<a href="https://colab.research.google.com/github/nirmal428/MachineLearning/blob/main/RandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Radom Forest Classifier with Pipeline and Hyperparameter Tuning

**Random Forest Classifier** is an ensemble machine learning algorithm used mainly for classification tasks.

**In short:**

**Based on decision trees :** It builds multiple decision trees during training.

**Ensemble method :** Combines predictions of many trees to improve accuracy.

**Works on majority voting :** Each tree gives a class, and the majority vote is chosen.

**Reduces overfitting :** By averaging multiple trees.

Handles missing values & large datasets well.

Good for both categorical & numerical data.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df=sns.load_dataset('tips')

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
df.isnull().sum()

,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [8]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [9]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [10]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
encoder=LabelEncoder()

In [13]:
df['time']=encoder.fit_transform(df['time'])

In [14]:
df['time']

,time
0,0
1,0
2,0
3,0
4,0
...,...
239,0
240,0
241,0
242,0


In [17]:
x=df.drop(labels=['time'],axis=1)
y=df['time']

In [18]:
x.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [19]:
y.head()

,time
0,0
1,0
2,0
3,0
4,0


In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# pipelene

We use Pipeline in sklearn to:

Combine multiple steps (e.g., preprocessing + model) into one workflow.

Avoid data leakage by applying transformations only on training data.

Make code cleaner & easier to manage.

Simplify cross-validation and grid search.

Ensure same preprocessing on both train and test data.

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  ## Handle Missin Values
from sklearn.preprocessing import StandardScaler  #feature scaling
from sklearn.preprocessing import OneHotEncoder  # categorical to numerical
from sklearn.compose import ColumnTransformer

In [22]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

Numerical Pipeline


In [23]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)


CATEGORICAL PIPELINE

In [35]:
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('onehotencoder',OneHotEncoder())
    ]
)

COMBINE BOTH PIPELINE

In [36]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [37]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [38]:
x_train=preprocessor.fit_transform(x_train)
x_test=preprocessor.transform(x_test)

In [39]:
from sklearn.ensemble import RandomForestClassifier

MODEL TRAINING AUTOMATION

In [40]:
models={
    'Random Forest':RandomForestClassifier()
}

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
def evaluate_model(x_train,x_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]

        #Train Model
        model.fit(x_train,y_train)

        #Predict Testing Data
        y_test_pred=model.predict(x_test)

        #Get Accuracy Score for test data
        test_model_score=accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]]=test_model_score

    return report

In [43]:
evaluate_model(x_train,x_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877}

In [44]:
classidier=RandomForestClassifier()

In [45]:
params={
    'max_depth':[3,5,10,None],
    'n_estimators':[10,20,50,100],
    'max_features':['auto','sqrt'],
    'min_samples_split':[2,4,6],
    'min_samples_leaf':[1,2,4]
}

In [46]:
from sklearn.model_selection import RandomizedSearchCV

In [47]:
cv=RandomizedSearchCV(classidier,param_distributions=params,n_iter=10,cv=5,verbose=2)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimat

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [3, 5, 10, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 4, 6],
                                        'n_estimators': [10, 20, 50, 100]},
                   verbose=2)

In [48]:
cv.best_params_

{'n_estimators': 20,
 'min_samples_split': 4,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 3}